# Metaclasses

Metaclasses provide customization of class instantiation.

## Creating a class the 'hard' way

In [56]:
def init(self, name):
    self.name = name

def say(self):
    print(self.name)
    
attrs = {
    '__init__': init,
    'say': say,
    'kingdom': 'catdom'
}

superclasses = tuple([object])
Cat = type('Cat', superclasses, attrs)

In [58]:
a = Cat('name')
a.say()

name


This is the same as:

In [1]:
class Cat(object):
    kingdom = 'catdom'
    def __init__(self, name):
        self.name = name
    def say(self):
        print(self.name)

## Workflow of metaclasses

In [13]:
import pprint as pp
import weakref

In [105]:
class MyMeta(type):
    
    # Optional
    @classmethod
    def __prepare__(cls, name, bases):
        '''
        Returns a dictionary or a mapping object...that is what is a namespace
        Creates the class namespace prior to the body of any class definition being processed.
        '''
        return super().__prepare__(name, bases)  # type.__prepare__(name, bases) returns a dictionary

    def __new__(cls, clsname, bases, clsdict):
        '''
        Creates an instance of the class MyMeta
        Called after the class body has been fully executed.
        Common that only __new__ or __init__ is defined in the metaclass, not both.
        '''
        
        print('MyMeta: __new__ start')
        print('arg0: {}'.format(cls))
        print('arg1: {}'.format(clsname))
        print('arg2: {}'.format(bases))
        print('arg3: {}'.format(pp.pformat(clsdict)))
        
        to_return = super().__new__(cls, clsname, bases, clsdict)
        to_return.attr_from_new = 'attr_from_new'
        
        print('returning: {}'.format(to_return))
        print('MyMeta: __new__ end')
        
        return to_return  # common mistake is to not return anything and just call the superclass

    
    def __init__(self, name, bases, dct):
        '''
        Initialize the class.
        
        self is an instance of the metaclass--i.e. a class
        '''
        print('MyMeta: __init__ start')
        print('arg0: {}'.format(self))
        print('arg1: {}'.format(name))
        print('arg2: {}'.format(bases))
        print('arg3: {}'.format(pp.pformat(dct)))

        self.attr_from_init = 'attr_from_init'
        
        super().__init__(name, bases, dct)
        
        print('MyMeta: __init__ end')
        
    def __call__(self, *args, **kwargs):
        print('MyMeta: __call__ start')
        print('arg0: {}'.format(self))
        print('args: {}'.format(pp.pformat(args)))
        print('kwargs: {}'.format(pp.pformat(kwargs)))
        to_return = super().__call__(*args, **kwargs)
        print('MyMeta: __call__ end')
        return to_return

In [106]:
class Foo(metaclass=MyMeta):
    
    FOO_CLASS_VARIABLE = 'BAR'
        
    def __new__(cls, *args, **kwargs):
        '''
        Creates an instance of the class Foo
        '''
        
        print('Foo: __new__ start')
        obj = super().__new__(cls, *args, **kwargs)
        obj.attr = 42
        print('arg0: {}'.format(cls))
        print('args: {}'.format(pp.pformat(args)))
        print('kwargs: {}'.format(pp.pformat(kwargs)))
        print('returning: {}'.format(obj))
        print('Foo: __new__ end')
        return obj
    
    def __init__(self):
        '''
        Initializes the instance.
        self is an instance of the class Foo
        '''
        print('Foo: __init__ start')
        print('arg0: {}'.format(str(self)))
        print('Foo: __init__ end')
    

MyMeta: __new__ start
arg0: <class '__main__.MyMeta'>
arg1: Foo
arg2: ()
arg3: {'FOO_CLASS_VARIABLE': 'BAR',
 '__call__': <function Foo.__call__ at 0x108e73048>,
 '__classcell__': <cell at 0x108e98c48: empty>,
 '__init__': <function Foo.__init__ at 0x108f57d08>,
 '__module__': '__main__',
 '__new__': <function Foo.__new__ at 0x108f57400>,
 '__qualname__': 'Foo'}
returning: <class '__main__.Foo'>
MyMeta: __new__ end
MyMeta: __init__ start
arg0: <class '__main__.Foo'>
arg1: Foo
arg2: ()
arg3: {'FOO_CLASS_VARIABLE': 'BAR',
 '__call__': <function Foo.__call__ at 0x108e73048>,
 '__classcell__': <cell at 0x108e98c48: MyMeta object at 0x7ff6ac2d7718>,
 '__init__': <function Foo.__init__ at 0x108f57d08>,
 '__module__': '__main__',
 '__new__': <function Foo.__new__ at 0x108f57400>,
 '__qualname__': 'Foo'}
MyMeta: __init__ end


In [107]:
a = Foo()

MyMeta: __call__ start
arg0: <class '__main__.Foo'>
args: ()
kwargs: {}
Foo: __new__ start
arg0: <class '__main__.Foo'>
args: ()
kwargs: {}
returning: <__main__.Foo object at 0x108f2ec18>
Foo: __new__ end
Foo: __init__ start
arg0: <__main__.Foo object at 0x108f2ec18>
Foo: __init__ end
MyMeta: __call__ end


In [132]:
Foo.attr_from_new

'attr_from_new'

In [131]:
Foo.attr_from_init

'attr_from_init'

In [35]:
Foo.FOO_CLASS_VARIABLE

'BAR'

In [40]:
a.attr_from_new

'attr_from_new'

In [41]:
a.attr_from_init

'attr_from_init'

In [42]:
a.attr

42

# Cached

In [100]:
class Cached(type):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
    
    def __call__(self, *args):
        
        print(args)
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

In [101]:
class DataModel(metaclass=Cached):
    pass

In [102]:
class FooDataModel(DataModel):
    def __init__(self, name):
        self.name = name

In [103]:
a = FooDataModel('Allen', foo=42)
b = FooDataModel('Allen', answer=42)

TypeError: __call__() got an unexpected keyword argument 'foo'

In [98]:
list(DataModel._Cached__cache.keys())

[]

In [99]:
a is b

True

In [76]:
weakref.ref(a)

<weakref at 0x108d84d18; to 'FooDataModel' at 0x108f67710>

In [67]:
import gc

In [73]:
gc.collect()

0

In [66]:
weakref.ref('42')

TypeError: cannot create weak reference to 'str' object

In [61]:
d = weakref.WeakKeyDictionary()

In [65]:
d[['42']] = 42

TypeError: cannot create weak reference to 'list' object

In [ ]:
d

In [ ]:
a

In [34]:
a = FooDataModel('hello')

In [35]:
id(a)

4445116848

In [36]:
b = FooDataModel('hello')

In [37]:
id(b)

4445197032

In [38]:
a is b

False